In [2]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    shared_drive_foldername = 'NUTRECON'
    root = '/content/drive/Shareddrives/{}/'.format(shared_drive_foldername)
    %pip install pandas==1.4.2
    data_path = _v_.dataPath
    print('Running Code in Colab')
except:
    root = 'D:/FV/Projects/NUTRECON/nutreconDrive/'
    data_path = "D:/FV/Projects/NUTRECON/Data/nutrecon/"
    print('Running Code locally')

import sys
sys.path.append(root + '1. Acquisition/notebooks')
# sys.path.append(root + '1. Acquisition/notebooks_psychophysics')


experimentCode = 'nutreMoneyTest'
from nutrecon import *

from random import shuffle, sample
import pandas as pd
import numpy as np

from glob import glob
from datetime import datetime

_v_ = Vars()

Running Code locally
Running Code locally


In [3]:
subject_nr = 0

subject_code = '{}{:03d}'.format(experimentCode, subject_nr)

print(subject_code)

nutreMoneyTest000


In [39]:
with open(data_path + "moneyTrialsTest_UniqueLotteries", "r") as fp:
  conditions = json.load(fp)
Subject_code = subject_code
section_fileID = 'day4_neuroEconOrder'
n_Lott_reps = 6
mixed_blocks = False

df = pd.DataFrame(conditions, columns = ['Trial Type', 'reference type','reference qt','reference p', 'lottery type', 'lottery qt','lottery p'])

fpath = '{}{}_{}_{}.json'.format(data_path, Subject_code, section_fileID, strTimestamp())
fileMatchingPattern = glob('{}{}*{}*.json'.format(data_path, Subject_code, section_fileID))
# if the file exists...
if check_MatchingPattern(data_path, Subject_code, section_fileID)[1]:
    # get the timestamp of file creation from the file name and provide the user that information
    timestamp = float(fileMatchingPattern[0][:-5].split('_')[-1])
    print('File already exists. Created on {}'.format(datetime.fromtimestamp(timestamp)))
    # open the json file as a dataframe
    df_final = pd.read_json(fileMatchingPattern[0], orient = 'index')
else:
    df = pd.DataFrame(conditions, columns = ['Trial Type', 'reference type','reference qt','reference p', 'lottery type', 'lottery qt','lottery p'])
    df['Trial ID'] = np.arange(len(df))
    df['reference flavor'] = ''
    df['reference shape'] = ''
    df['lottery flavor'] = ''
    df['lottery shape'] = ''
    # each lotery option is repeated 6 times (in the whole task)
    blocks = []
    for p in range(n_Lott_reps):
        blocks += [df.sample(frac = 1).values.tolist()]        

    df_final = pd.DataFrame()  # new empty dataframe
    block = 0
    for b in blocks: # each block 
        df_tmp = pd.DataFrame(b, columns=df.columns)
        # df_tmp.columns = same_df.columns
        df_tmp['block'] = block
        #df_final = df_final.append(df_tmp)
        df_final = pd.concat([df_final, df_tmp]) 
        block += 1

    df_final.reset_index(drop = True, inplace = True)
    df_final.to_json(fpath, orient='index')
    # save the file as json in the proper path
    # df_final.to_json(fpath, orient='index')
  
#   #return the task trials as a dataframe, as well as the C+ and C- flavours
#   return df_final, cPlus_flavor, cMinus_flavor